In [ ]:
# # # Mount the drive into google colab

# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)

In [ ]:
!pip install "numpy<2.0.0"
!apt-get install unrar
!pip install grad-cam
!pip install opencv-python-headless
!pip install imgaug
# !pip uninstall -y torch torchvision torchaudio
# !pip install torch torchvision torchaudio
!python -m pip install lightning


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unrar is already the newest version (1:6.1.5-1ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.0/819.0 kB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 961.5/961.5 kB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.1/823.1 kB 51.1 MB/s eta 0:00:00


In [ ]:
!unrar x "/content/EBHI-SEG.rar" "/content"


UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from /content/EBHI-SEG.rar

Creating    /content/EBHI-SEG                                         OK
Creating    /content/EBHI-SEG/Adenocarcinoma                          OK
Creating    /content/EBHI-SEG/Adenocarcinoma/test                     OK
Extracting  /content/EBHI-SEG/Adenocarcinoma/test/GT2001837-1-400-002.png       0%  OK 
Extracting  /content/EBHI-SEG/Adenocarcinoma/test/GT2001849-1-400-001.png       0%  OK 
Extracting  /content/EBHI-SEG/Adenocarcinoma/test/GT2001849-1-400-012.png       0%  OK 
Extracting  /content/EBHI-SEG/Adenocarcinoma/test/GT2012165-1-400-006.png       0%  OK 
Extracting  /content/EBHI-SEG/Adenocarcinoma/test/GT2012223-4-400-002.png       0%  OK 
Extracting  /content/EBHI-SEG/Adenocarcinoma/test/GT2012324-1-400-011.png       0%  OK 
Extracting  /content/EBHI-SEG/Adenocarcinoma/test/GT2012324-1-400-014.png       

In [ ]:
import os
#Thiết lập thư mục làm việc cho toàn bộ code
# Sửa đường dẫn này cho phù hợp với vị trí bạn lưu dataset trên Drive
WORKING_DIR = '/content'

# Đường dẫn đến thư mục chứa dataset sau khi giải nén
DATASET_DIR = os.path.join(WORKING_DIR, "EBHI-SEG")

In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split
# Tỷ lệ chia train/val/test
TRAIN_RATIO = 0.7
VAL_RATIO = 0.15
TEST_RATIO = 0.15

# Dataset selection
DATASET_CHOICE = DATASET_DIR

# Create a definition dictionary
dict_img_class = {}

try:
    for i, class_name in enumerate(os.listdir(DATASET_CHOICE)):
        dict_img_class[class_name] = i

    class_name = list(dict_img_class.keys())

    #print("Dictionary ánh xạ class:", dict_img_class)
except FileNotFoundError:
    print(f"Lỗi: Không tìm thấy thư mục dataset tại {DATASET_CHOICE}")

try:
    # Chia ngẫu nhiên dữ liệu vào các thư mục train/val/test
    for class_name_item in os.listdir(DATASET_DIR):
        if class_name_item.startswith("."):
            continue # bỏ qua các file ẩn
        class_dir = os.path.join(DATASET_DIR, class_name_item)
        image_dir = os.path.join(class_dir, "image") # Đường dẫn đến folder image
        label_dir = os.path.join(class_dir, "label") # Đường dẫn đến folder label
        if os.path.isdir(image_dir):
            # Lấy danh sách tất cả các file ảnh trong thư mục image_dir
            list_all_img_full_path = [os.path.join(image_dir, file_name) for file_name in os.listdir(image_dir) if not file_name.startswith('.')]
            # Chia ngẫu nhiên
            train_files, test_val_files = train_test_split(list_all_img_full_path, test_size=(VAL_RATIO + TEST_RATIO), random_state=42)
            val_files, test_files = train_test_split(test_val_files, test_size=TEST_RATIO/(VAL_RATIO + TEST_RATIO), random_state=42)

            # Tạo các thư mục train/val/test nếu chưa tồn tại
            for type_dir in ["train", "test", "val"]:
                target_dir = os.path.join(class_dir, type_dir)
                if not os.path.exists(target_dir):
                    os.makedirs(target_dir)

            # Di chuyển các file vào các thư mục tương ứng
            # Thay vì di chuyển, ta sẽ copy file sang các thư mục tương ứng
            for file in train_files:
                shutil.copy(file, os.path.join(class_dir, "train", os.path.basename(file)))
            for file in val_files:
                shutil.copy(file, os.path.join(class_dir, "val", os.path.basename(file)))
            for file in test_files:
                shutil.copy(file, os.path.join(class_dir, "test", os.path.basename(file)))
            # Xóa thư mục label
            if os.path.exists(label_dir):
               shutil.rmtree(label_dir)
            if os.path.exists(image_dir):
               shutil.rmtree(image_dir)
    print("Đã chia ngẫu nhiên dữ liệu vào các thư mục train/val/test thành công!")

except FileNotFoundError:
    print(f"Lỗi: Không tìm thấy thư mục dataset tại {DATASET_DIR}")

Đã chia ngẫu nhiên dữ liệu vào các thư mục train/val/test thành công!


In [ ]:
# import os
# import cv2
# import numpy as np
# import random

# def rotate_image(image, angle):
#     """Xoay ảnh một góc ngẫu nhiên."""
#     height, width = image.shape[:2]
#     center = (width / 2, height / 2)
#     rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
#     rotated_image = cv2.warpAffine(image, rotation_matrix, (width, height))
#     return rotated_image

# def augment_images_by_ratio(data_dir):
#     """Tăng cường dữ liệu bằng cách xoay ảnh trong thư mục train và val theo tỉ lệ cho trước."""

#     # Tỉ lệ tăng cường cho từng lớp (class)
#     augmentation_ratios = {
#         "High-grade IN": 2600,
#         "Normal": 5200,
#         "Serrated adenoma": 5200,
#         "Low-grade IN": 1400,
#         "Adenocarcinoma": 1200,
#         "Polyp": 1400
#     }

#     for class_name in os.listdir(data_dir):
#         class_dir = os.path.join(data_dir, class_name)
#         if not os.path.isdir(class_dir):
#             continue

#         if class_name not in augmentation_ratios:
#             print(f"Lớp {class_name} không có trong danh sách tỉ lệ tăng cường. Bỏ qua.")
#             continue

#         ratio = augmentation_ratios[class_name]

#         for subset in ["train", "val"]:
#             subset_dir = os.path.join(class_dir, subset)
#             if not os.path.isdir(subset_dir):
#                 continue

#             images = [f for f in os.listdir(subset_dir) if os.path.isfile(os.path.join(subset_dir, f)) and f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif'))]
#             num_images = len(images)
#             #print(num_images)
#             num_augmentations_needed = int(num_images * ratio / 100)
#             #print(num_augmentations_needed)
#             if num_augmentations_needed <= 0:
#                 print(f"Lớp {class_name} trong {subset} đã đủ số lượng ảnh, không cần tăng cường thêm.")
#                 continue

#             print(f"Tăng cường {num_augmentations_needed} ảnh cho lớp {class_name} trong {subset}...")

#             # Lặp cho đến khi đủ số lượng ảnh cần tăng cường
#             augment_count = 0
#             image_index = 0
#             while augment_count < num_augmentations_needed:
#               image_name = images[image_index % num_images]  # Lấy ảnh gốc để xoay
#               image_path = os.path.join(subset_dir, image_name)
#               original_image = cv2.imread(image_path)

#               if original_image is None:
#                   print(f"Lỗi: Không thể đọc ảnh tại {image_path}")
#                   image_index +=1
#                   continue

#               original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)
#               # Xoay ảnh 1 góc ngẫu nhiên từ -20 đến 20 độ
#               angle = random.uniform(-20, 20)
#               rotated_image = rotate_image(original_image, angle)

#               if rotated_image is not None:
#                 # Tạo tên file mới cho ảnh đã xoay
#                 file_name_new = os.path.splitext(image_name)[0] + f"_rotated_{augment_count}.jpg"
#                 output_path = os.path.join(subset_dir, file_name_new)
#                 cv2.imwrite(output_path, cv2.cvtColor(np.uint8(rotated_image), cv2.COLOR_RGB2BGR))
#                 augment_count += 1
#               else:
#                 print(f"Không thể xoay ảnh {image_path}")

#               image_index += 1

#     print("Hoàn tất quá trình tăng cường dữ liệu!")

# # Gọi hàm để thực hiện tăng cường dữ liệu
# augment_images_by_ratio(DATASET_DIR)

In [ ]:
import warnings
from argparse import ArgumentParser
from pathlib import Path
from imgaug import augmenters as iaa

warnings.filterwarnings("ignore")
import numpy as np
import pytorch_lightning as pl
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from torch.optim import SGD, Adam
from torch.utils.data import DataLoader
from torchmetrics import Accuracy
from torchvision import transforms
from torchvision.datasets import ImageFolder
#from torchvision.models.swin_transformer.SwinTransformer


# Here we define a new class to turn the ResNet model that we want to use as a feature extractor
# into a pytorch-lightning module so that we can take advantage of lightning's Trainer object.
# We aim to make it a little more general by allowing users to define the number of prediction classes.
class ResNetClassifier(pl.LightningModule):
    resnets = {
        18: models.resnet18,
        34: models.resnet34,
        50: models.resnet50,
        101: models.resnet101,
        152: models.resnet152,
    }
    optimizers = {"adam": Adam, "sgd": SGD}

    def __init__(
        self,
        num_classes,
        resnet_version,
        train_path,
        val_path,
        test_path=None,
        optimizer="adam",
        lr=1e-3,
        batch_size=16,
        transfer=True,
        tune_fc_only=True,
    ):
        super().__init__()

        self.num_classes = num_classes
        self.train_path = train_path
        self.val_path = val_path
        self.test_path = test_path
        self.lr = lr
        self.batch_size = batch_size

        self.optimizer = self.optimizers[optimizer]
        # instantiate loss criterion
        self.loss_fn = (
            nn.BCEWithLogitsLoss() if num_classes == 1 else nn.CrossEntropyLoss()
        )
        # create accuracy metric
        self.acc = Accuracy(
            task="binary" if num_classes == 1 else "multiclass", num_classes=num_classes
        )
        # Using a pretrained ResNet backbone
        self.resnet_model = self.resnets[resnet_version](pretrained=transfer)
        # Replace old FC layer with Identity so we can train our own
        linear_size = list(self.resnet_model.children())[-1].in_features
        # replace final layer for fine tuning
        self.resnet_model.fc = nn.Linear(linear_size, num_classes)

        if tune_fc_only:  # option to only tune the fully-connected layers
            for child in list(self.resnet_model.children())[:-1]:
                for param in child.parameters():
                    param.requires_grad = False

    def forward(self, X):
        return self.resnet_model(X)

    def configure_optimizers(self):
        return self.optimizer(self.parameters(), lr=self.lr)

    def _step(self, batch):
        x, y = batch
        preds = self(x)

        if self.num_classes == 1:
            preds = preds.flatten()
            y = y.float()

        loss = self.loss_fn(preds, y)
        acc = self.acc(preds, y)
        return loss, acc

    def _dataloader(self, data_path, shuffle=False):
        img_size = 500
        # values here are specific to pneumonia dataset and should be updated for custom data
        tramsform = None
        if "train" in data_path:
          transform = transforms.Compose(
              [
                  np.asarray,
                  iaa.Sequential([
                    iaa.pillike.EnhanceColor(),
                    iaa.color.AddToHue((-30,30)),
                    iaa.PerspectiveTransform(scale=(0, 0.15)),
                    iaa.Crop(percent=(0, 0.15), sample_independently=False),
                    iaa.GaussianBlur(sigma=(0.0, 0.1)),

                    iaa.MultiplyBrightness(mul=(0.65, 1.35)),
                    iaa.Resize({"height": img_size, "width": img_size})
                  ]).augment_image,
                  np.copy,
                  transforms.ToTensor(),
                  transforms.Normalize((0.48232,), (0.23051,)),
              ]
          )

        else:
          transform = transforms.Compose(
              [
                  np.asarray,
                  iaa.Sequential([
                    iaa.Resize({"height": img_size, "width": img_size})
                  ]).augment_image,
                  np.copy,
                  transforms.ToTensor(),
                  transforms.Normalize((0.48232,), (0.23051,)),
              ]
          )

        img_folder = ImageFolder(data_path, transform=transform)

        return DataLoader(img_folder, batch_size=self.batch_size, shuffle=shuffle)

    def train_dataloader(self):
        return self._dataloader(self.train_path, shuffle=True)

    def training_step(self, batch, batch_idx):
        loss, acc = self._step(batch)
        # perform logging
        self.log(
            "train_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True
        )
        self.log(
            "train_acc", acc, on_step=True, on_epoch=True, prog_bar=True, logger=True
        )
        return loss

    def val_dataloader(self):
        return self._dataloader(self.val_path)

    def validation_step(self, batch, batch_idx):
        loss, acc = self._step(batch)
        # perform logging
        self.log("val_loss", loss, on_epoch=True, prog_bar=False, logger=True)
        self.log("val_acc", acc, on_epoch=True, prog_bar=True, logger=True)

    def test_dataloader(self):
        return self._dataloader(self.test_path)

    def test_step(self, batch, batch_idx):
        loss, acc = self._step(batch)
        # perform logging
        self.log("test_loss", loss, on_step=True, prog_bar=True, logger=True)
        self.log("test_acc", acc, on_step=True, prog_bar=True, logger=True)



In [ ]:

def main( num_classes=6,
    resnet_version=18,
    train_path='./train',
    val_path='./val',
    test_path='./test',
    optimizer='adam',
    lr=1e-3,
    batch_size=16,
    transfer=True,
    tune_fc_only=False,
    num_epochs = 50,
    save_path = '.',
    mixed_precision = False):


    # # Instantiate Model
    model = ResNetClassifier(
        num_classes=num_classes,
        resnet_version=resnet_version,
        train_path=train_path,
        val_path=val_path,
        test_path=test_path,
        optimizer=optimizer,
        lr=lr,
        batch_size=batch_size,
        transfer=transfer,
        tune_fc_only=tune_fc_only,
    )

    save_path = save_path if save_path is not None else "./models"
    checkpoint_callback = pl.callbacks.ModelCheckpoint(
        dirpath=save_path,
        filename="resnet-model-{epoch}-{val_loss:.2f}-{val_acc:0.2f}",
        monitor="val_loss",
        save_top_k=3,
        mode="min",
        save_last=True,
    )

    stopping_callback=pl.callbacks.EarlyStopping(monitor="val_loss", mode="min", patience=20)
    # Instantiate lightning trainer and train model
    trainer_args = {
        "accelerator": "cuda",
        "max_epochs": num_epochs,
        "callbacks": [checkpoint_callback, stopping_callback],
        "precision": 16 if mixed_precision else 32,
    }
    trainer = pl.Trainer(**trainer_args, )

    trainer.fit(model)

    if test_path:
        trainer.test(model)
    # Save trained model weights
    torch.save(trainer.model.resnet_model.state_dict(), save_path + "/trained_model.pt")

    return model

In [ ]:
!mkdir train
!mkdir val
!mkdir test

!mkdir train/c1


!mkdir train/c2
!mkdir train/c3
!mkdir train/c4
!mkdir train/c5
!mkdir train/c6

!mkdir val/c1
!mkdir val/c2
!mkdir val/c3
!mkdir val/c4
!mkdir val/c5
!mkdir val/c6

!mkdir test/c1
!mkdir test/c2
!mkdir test/c3
!mkdir test/c4
!mkdir test/c5
!mkdir test/c6
!mv /content/EBHI-SEG/Adenocarcinoma/train/* train/c1
!mv /content/EBHI-SEG/High-grade\ IN/train/* train/c2
!mv /content/EBHI-SEG/Low-grade\ IN/train/* train/c3
!mv /content/EBHI-SEG/Normal/train/* train/c4
!mv /content/EBHI-SEG/Polyp/train/* train/c5
!mv /content/EBHI-SEG/Serrated\ adenoma/train/* train/c6

!mv /content/EBHI-SEG/Adenocarcinoma/test/* test/c1
!mv /content/EBHI-SEG/High-grade\ IN/test/* test/c2
!mv /content/EBHI-SEG/Low-grade\ IN/test/* test/c3
!mv /content/EBHI-SEG/Normal/test/* test/c4
!mv /content/EBHI-SEG/Polyp/test/* test/c5
!mv /content/EBHI-SEG/Serrated\ adenoma/test/* test/c6

!mv /content/EBHI-SEG/Adenocarcinoma/val/* val/c1
!mv /content/EBHI-SEG/High-grade\ IN/val/* val/c2
!mv /content/EBHI-SEG/Low-grade\ IN/val/* val/c3
!mv /content/EBHI-SEG/Normal/val/* val/c4
!mv /content/EBHI-SEG/Polyp/val/* val/c5
!mv /content/EBHI-SEG/Serrated\ adenoma/val/* val/c6

In [ ]:
model = main()

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 202MB/s]
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name         | Type               | Params | Mode 
------------------------------------------------------------
0 | loss_fn      | CrossEntropyLoss   | 0      | train
1 | acc          | MulticlassAccuracy | 0      | train
2 | resnet_model | ResNet             | 11.2 M | train
------------------------------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.718    Total 

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:
Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined

In [ ]:
from argparse import ArgumentParser
from pathlib import Path
from imgaug import augmenters as iaa

import numpy as np
import pytorch_lightning as pl
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from torch.optim import SGD, Adam
from torch.utils.data import DataLoader
from torchmetrics import Accuracy, Precision, Recall
from torchmetrics.classification import MulticlassF1Score, MulticlassRecall, MulticlassPrecision, BinaryF1Score, BinaryRecall, BinaryPrecision
from torchvision import transforms
from torchvision.datasets import ImageFolder

from torchmetrics import Metric



# Here we define a new class to turn the ResNet model that we want to use as a feature extractor
# into a pytorch-lightning module so that we can take advantage of lightning's Trainer object.
# We aim to make it a little more general by allowing users to define the number of prediction classes.
class ResNetClassifier(pl.LightningModule):
    resnets = {
        18: models.resnet18,
        34: models.resnet34,
        50: models.resnet50,
        101: models.resnet101,
        152: models.resnet152,
    }
    optimizers = {"adam": Adam, "sgd": SGD}

    def __init__(
        self,
        num_classes,
        resnet_version,
        train_path,
        val_path,
        test_path=None,
        optimizer="adam",
        lr=1e-3,
        batch_size=16,
        transfer=True,
        tune_fc_only=True,
        name="<just give the path file here>"
    ):
        super().__init__()

        self.num_classes = num_classes
        self.train_path = train_path
        self.val_path = val_path
        self.test_path = test_path
        self.lr = lr
        self.batch_size = batch_size
        self.name = name
        self.optimizer = self.optimizers[optimizer]
        self.count = 0
        # instantiate loss criterion
        self.loss_fn = (
            nn.BCEWithLogitsLoss() if num_classes == 1 else nn.CrossEntropyLoss()
        )
        # create accuracy metric
        self.acc = Accuracy(
            task="binary" if num_classes == 1 else "multiclass", num_classes=num_classes
        )
        self.acc_top_2 = Accuracy(
            task="binary" if num_classes == 1 else "multiclass", num_classes=num_classes, top_k=2
        )
        self.acc_top_3 = Accuracy(
            task="binary" if num_classes == 1 else "multiclass", num_classes=num_classes, top_k=3
        )

        self.prec = MulticlassPrecision(num_classes=num_classes, #average=None, #multidim_average="samplewise"
        )
        self.prec_top_2 = MulticlassPrecision(num_classes=num_classes, top_k=2, #average=None, multidim_average="samplewise"
                                              )
        self.prec_top_3 = MulticlassPrecision( num_classes=num_classes, top_k=3, #average=None, #multidim_average="samplewise"
                                              )
        self.recall = MulticlassRecall(num_classes=num_classes, #average=None, #multidim_average="samplewise"
        )
        self.recall_top_2 = MulticlassRecall(num_classes=num_classes, top_k=2, #average=None, #multidim_average="samplewise"
        )
        self.recall_top_3 = MulticlassRecall(num_classes=num_classes, top_k=3, #average=None, #multidim_average="samplewise"
                                             )

        self.f1 = MulticlassF1Score(num_classes=num_classes,  #multidim_average="samplewise"
        )
        self.f1_top_2 = MulticlassF1Score(num_classes=num_classes, top_k=2, #average=None,  #multidim_average="samplewise"
                                          )
        self.f1_top_3 = MulticlassF1Score(num_classes=num_classes, top_k=3, #average=None,  #multidim_average="samplewise"
                                          )
        # Using a pretrained ResNet backbone
        self.resnet_model = self.resnets[resnet_version](pretrained=transfer)
        # Replace old FC layer with Identity so we can train our own
        linear_size = list(self.resnet_model.children())[-1].in_features
        # replace final layer for fine tuning
        self.resnet_model.fc = nn.Linear(linear_size, num_classes)

        if tune_fc_only:  # option to only tune the fully-connected layers
            for child in list(self.resnet_model.children())[:-1]:
                for param in child.parameters():
                    param.requires_grad = False

    def forward(self, X):
        return self.resnet_model(X)

    def configure_optimizers(self):
        return self.optimizer(self.parameters(), lr=self.lr)

    def _step(self, batch):
        x, y = batch
        preds = self(x)

        if self.num_classes == 1:
            preds = preds.flatten()
            y = y.float()

        loss = self.loss_fn(preds, y)
        acc = self.acc(preds, y)
        acc_top_2 = self.acc_top_2(preds, y)

        acc_top_3 = self.acc_top_3(preds, y)

        apreds = preds
        ay = y

        prec = self.prec(apreds, ay)
        prec_top_2 = self.prec_top_2(apreds, ay)
        prec_top_3 =self.prec_top_3(apreds, ay)

        recall = self.recall(apreds, ay)
        recall_top_2 = self.recall_top_2(apreds, ay)
        recall_top_3 = self.recall_top_3(apreds, ay)

        f1 = self.f1(apreds, ay)
        f1_top_2 = self.f1_top_2(apreds, ay)
        f1_top_3 = self.f1_top_3(apreds, ay)

        print(preds.shape, y.shape)
        self.count = self.count + 1
        np.save(self.name + str(self.count) + "pred.npz", preds.cpu().detach().numpy())
        np.save(self.name + str(self.count) + "label.npz", y.cpu().detach().numpy())
        return loss, acc, acc_top_2, acc_top_3, prec, prec_top_2, prec_top_3, recall, recall_top_2, recall_top_3, f1, f1_top_2, f1_top_3

    def _dataloader(self, data_path, shuffle=False):
        img_size = 500
        # values here are specific to pneumonia dataset and should be updated for custom data
        tramsform = None
        if "train" in data_path:
          transform = transforms.Compose(
              [
                  np.asarray,
                  iaa.Sequential([
                    iaa.pillike.EnhanceColor(),
                    iaa.color.AddToHue((-30,30)),
                    iaa.PerspectiveTransform(scale=(0, 0.15)),
                    iaa.Crop(percent=(0, 0.15), sample_independently=False),
                    iaa.GaussianBlur(sigma=(0.0, 0.1)),

                    iaa.MultiplyBrightness(mul=(0.65, 1.35)),
                    iaa.Resize({"height": img_size, "width": img_size})
                  ]).augment_image,
                  np.copy,
                  transforms.ToTensor(),
                  transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), # Changed Normalization
              ]
          )

        else:
          transform = transforms.Compose(
              [
                  np.asarray,
                  iaa.Sequential([
                    iaa.Resize({"height": img_size, "width": img_size})
                  ]).augment_image,
                  np.copy,
                  transforms.ToTensor(),
                  transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), # Changed Normalization
              ]
          )

        img_folder = ImageFolder(data_path, transform=transform)

        return DataLoader(img_folder, batch_size=self.batch_size, shuffle=shuffle)

    def train_dataloader(self):
        return self._dataloader(self.train_path, shuffle=True)

    def training_step(self, batch, batch_idx):
        loss, acc = self._step(batch)
        # perform logging
        self.log(
            "train_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True
        )
        self.log(
            "train_acc", acc, on_step=True, on_epoch=True, prog_bar=True, logger=True
        )
        return loss

    def val_dataloader(self):
        return self._dataloader(self.val_path)

    def validation_step(self, batch, batch_idx):
        loss, acc = self._step(batch)
        # perform logging
        self.log("val_loss", loss, on_epoch=True, prog_bar=False, logger=True)
        self.log("val_acc", acc, on_epoch=True, prog_bar=True, logger=True)

    def test_dataloader(self):
        return self._dataloader(self.test_path)

    def test_step(self, batch, batch_idx):
        loss, acc, acc_top_2, acc_top_3, prec, prec_top_2, prec_top_3, recall, recall_top_2, recall_top_3, f1, f1_top_2, f1_top_3 = self._step(batch)
        # perform logging
        self.log("test_loss", loss, on_step=True, prog_bar=True, logger=True)
        self.log("test_acc", acc, on_step=True, prog_bar=True, logger=True)
        self.log("test_acc_top_2", acc_top_2, on_step=True, prog_bar=True, logger=True)
        self.log("test_acc_top_3", acc_top_3, on_step=True, prog_bar=True, logger=True)

        self.log("test_acc", acc, on_step=True, prog_bar=True, logger=True)
        self.log("test_acc_top_2", acc_top_2, on_step=True, prog_bar=True, logger=True)
        self.log("test_acc_top_3", acc_top_3, on_step=True, prog_bar=True, logger=True)


        self.log("test_prec", prec, on_step=True, prog_bar=True, logger=True)
        self.log("test_prec_top_2", prec_top_2, on_step=True, prog_bar=True, logger=True)
        self.log("test_prec_top_3", prec_top_3, on_step=True, prog_bar=True, logger=True)

        self.log("test_recall", recall, on_step=True, prog_bar=True, logger=True)
        self.log("test_recall_top_2", recall_top_2, on_step=True, prog_bar=True, logger=True)
        self.log("test_recallc_top_3", recall_top_3, on_step=True, prog_bar=True, logger=True)

        self.log("test_f1", f1, on_step=True, prog_bar=True, logger=True)
        self.log("test_f1_top_2", f1_top_2, on_step=True, prog_bar=True, logger=True)
        self.log("test_f1_top_3", f1_top_3, on_step=True, prog_bar=True, logger=True)

def run_test(filename, resnet_version):
  num_classes=6
  resnet_version=resnet_version
  train_path='./train'
  val_path='./val'
  test_path='./test'
  optimizer='adam'
  lr=1e-3
  batch_size=64
  transfer=True
  tune_fc_only=False
  num_epochs = 2
  save_path = '.'
  mixed_precision = False
  model = ResNetClassifier.load_from_checkpoint(filename,
        num_classes=6,
        resnet_version=resnet_version,
        train_path=train_path,
        val_path=val_path,
        test_path=test_path,
        optimizer=optimizer,
        lr=lr,
        batch_size=batch_size,
        transfer=transfer,
        tune_fc_only=tune_fc_only,
        name=filename
    )

  # disable randomness, dropout, etc...
  model.eval()



  checkpoint_callback = pl.callbacks.ModelCheckpoint(
      dirpath=save_path,
      filename="resnet-model-{epoch}-{val_loss:.2f}-{val_acc:0.2f}",
      monitor="val_loss",
      save_top_k=3,
      mode="min",
      save_last=True,
  )

  stopping_callback=pl.callbacks.EarlyStopping(monitor="val_loss", mode="min")
  #Instantiate lightning trainer and train model
  trainer_args = {
      "accelerator": "cuda",
      "max_epochs": num_epochs,
      "callbacks": [checkpoint_callback, stopping_callback],
      "precision": 16 if mixed_precision else 32,
  }
  trainer = pl.Trainer(**trainer_args, )

  trainer.test(model)
  import os
list_file = os.listdir("./")
for i in list_file:
  if ".ckpt" in i and ".npz" not in i:
    number = 18
    if "resnet50" in i:
      number = 50
    elif "resnet34" in i:
      number = 34
    print(i, number)
    run_test(i, number)

resnet18-model-epoch=3-val_loss=0.55-val_acc=0.83.ckpt 18


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

torch.Size([64, 6]) torch.Size([64])
torch.Size([64, 6]) torch.Size([64])
torch.Size([64, 6]) torch.Size([64])
torch.Size([64, 6]) torch.Size([64])
torch.Size([64, 6]) torch.Size([64])
torch.Size([17, 6]) torch.Size([17])


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│      test_acc_epoch       │    0.8991097807884216     │
│   test_acc_top_2_epoch    │    0.9762611389160156     │
│   test_acc_top_3_epoch    │    0.9910979270935059     │
│       test_f1_epoch       │    0.4777697026729584     │
│    test_f1_top_2_epoch    │    0.7085807919502258     │
│    test_f1_top_3_epoch    │    0.8829910755157471     │
│      test_loss_epoch      │    0.33174383640289307    │
│      test_prec_epoch      │    0.5136367678642273     │
│   test_prec_top_2_epoch   │    0.9960982799530029     │
│   test_prec_top_3_epoch   │    0.9978898763656616     │
│     test_recall_epoch     │    0.4550110995769501     │
│  test_recall_top_2_epoch  │    0.9631388187408447     │
│ test_recallc_top_3_epoch  │    0.9894822835922241     │
└───────────────────────────┴───────────────────────────┘